# Análisis Técnico básico para Mercados Financieros


---



Este documento presenta un análisis técnico básico para mercados financieros utilizando Python. 
El objetivo es proporcionar una herramienta para visualizar datos históricos de precios de df, calcular indicadores técnicos populares y generar gráficos que ayuden a identificar tendencias y patrones en el mercado.

---



## Importar Librerías Necesarias


---


### Importar las librerías necesarias, incluyendo pandas, plotly, datetime y textwrap.


In [195]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime
import textwrap
import snowflake.connector

## Solicitar Fechas y Tickers al Usuario


---


### Ingrese las fechas de inicio y fin para el análisis, así como los tickers (símbolos) de las df que se analizarán.


In [196]:
# Solicitar fechas y tickers al usuario
fecha_inicio = "2020-01-01"
fecha_fin = "2024-12-31"
tickers = ["BBVA.MC"]

In [197]:
# Conectar a Snowflake
conn = snowflake.connector.connect(
    user='TFMGRUPO4',
    password='TFMgrupo4ucm01_01#',
    account='WYNIFVB-YE01854',
    warehouse='COMPUTE_WH',
    database='YAHOO_FINANCE',
    schema='MACHINE_LEARNING',
    role='ACCOUNTADMIN'
)


In [198]:
# Mostrar tickers disponibles
query_tickers = """
    SELECT DISTINCT TICKER 
    FROM TICKERS_INDEX
    ORDER BY TICKER
"""
cursor = conn.cursor()
cursor.execute(query_tickers)
available_tickers = pd.DataFrame(cursor.fetchall(), columns=[col[0] for col in cursor.description])
display(available_tickers)


,TICKER
0,0JSJ.L
1,0JYA.L
2,0K34.L
3,0RIH.L
4,0RPW.L
...,...
1348,ZIOC.L
1349,ZOO.L
1350,ZUGN.SW
1351,ZURN.SW


## Obtener Datos de las df


---


### Descargamos los datos historicos para el ticker  y el periodo seleccionado.


In [199]:
# Consulta para extraer precios
query_data = f"""
    SELECT TICKER, FECHA, CLOSE, OPEN, HIGH, LOW, VOLUME
    FROM TICKERS_INDEX
    WHERE FECHA BETWEEN '{fecha_inicio}' AND '{fecha_fin}'
    AND TICKER IN ({','.join([f"'{s}'" for s in tickers])})
    ORDER BY FECHA
"""
cursor.execute(query_data)
df = pd.DataFrame(cursor.fetchall(), columns=[col[0] for col in cursor.description])

cursor.close()
conn.close()

df

,TICKER,FECHA,CLOSE,OPEN,HIGH,LOW,VOLUME
0,BBVA.MC,2020-01-02,5.110000134,5.031000137,5.124000072,5.020999908,11845254
1,BBVA.MC,2020-01-03,5.053999901,5.078000069,5.078000069,5.015999794,13399207
2,BBVA.MC,2020-01-06,5.032000065,5.000000000,5.039000034,4.943500042,11340344
3,BBVA.MC,2020-01-07,5.010000229,5.051000118,5.078999996,4.982999802,10427840
4,BBVA.MC,2020-01-08,5.058000088,4.968500137,5.068999767,4.960999966,12830856
...,...,...,...,...,...,...,...
1275,BBVA.MC,2024-12-20,9.302000046,9.286000252,9.317999840,9.131999969,34278281
1276,BBVA.MC,2024-12-23,9.300000191,9.328000069,9.366000175,9.206000328,3731184
1277,BBVA.MC,2024-12-24,9.265999794,9.289999962,9.319999695,9.251999855,1616478
1278,BBVA.MC,2024-12-27,9.385999680,9.239999771,9.395999908,9.239999771,4335543


In [200]:
# Convertir las columnas numéricas de 'df' a tipo float y 'VOLUME' a int
df['CLOSE'] = df['CLOSE'].astype(float)
df['OPEN'] = df['OPEN'].astype(float)
df['HIGH'] = df['HIGH'].astype(float)
df['LOW'] = df['LOW'].astype(float)
df['VOLUME'] = df['VOLUME'].astype(int)


In [201]:
df.dtypes

TICKER            object
FECHA     datetime64[ns]
CLOSE            float64
OPEN             float64
HIGH             float64
LOW              float64
VOLUME             int32
dtype: object

In [202]:
df.columns

Index(['TICKER', 'FECHA', 'CLOSE', 'OPEN', 'HIGH', 'LOW', 'VOLUME'], dtype='object')

In [203]:
# Filtrar solo el ticker que necesitas y establecer FECHA como índice
acciones = df[df['TICKER'] == tickers[0]].set_index('FECHA').drop('TICKER', axis=1)
acciones.head()

,CLOSE,OPEN,HIGH,LOW,VOLUME
FECHA,,,,,
2020-01-02,5.110,5.0310,5.124,5.0210,11845254
2020-01-03,5.054,5.0780,5.078,5.0160,13399207
2020-01-06,5.032,5.0000,5.039,4.9435,11340344
2020-01-07,5.010,5.0510,5.079,4.9830,10427840
2020-01-08,5.058,4.9685,5.069,4.9610,12830856


In [204]:
# Pivotear para obtener precios por columna de ticker
acciones_cierre = df.pivot(index='FECHA', columns='TICKER', values='CLOSE')
acciones_cierre .head()

TICKER,BBVA.MC
FECHA,
2020-01-02,5.110
2020-01-03,5.054
2020-01-06,5.032
2020-01-07,5.010
2020-01-08,5.058


In [205]:
#dataframe de los cierres de acciones
acciones_cierre

TICKER,BBVA.MC
FECHA,
2020-01-02,5.110
2020-01-03,5.054
2020-01-06,5.032
2020-01-07,5.010
2020-01-08,5.058
...,...
2024-12-20,9.302
2024-12-23,9.300
2024-12-24,9.266


## Calcular RSI


---


### La función calcular_rsi tiene como objetivo calcular el Índice de Fuerza Relativa (RSI) de una serie de datos de precios (usualmente los precios de cierre de una acción). El RSI es un indicador técnico de momentum que compara la magnitud de las ganancias recientes con la magnitud de las pérdidas recientes para determinar las condiciones de sobrecompra o sobreventa en el precio de una acción o activo.

In [206]:
# Definir una función para calcular el Índice de Fuerza Relativa (RSI)
def calcular_rsi(datos, ventana):
    delta = datos.diff()
    ganancia = (delta.where(delta > 0, 0)).rolling(window=ventana).mean()
    perdida = (-delta.where(delta < 0, 0)).rolling(window=ventana).mean()
    rs = ganancia / perdida
    rsi = 100 - (100 / (1 + rs))
    return rsi

## Calcular MACD


---


### La función calcular_macd tiene como objetivo calcular la Convergencia/Divergencia de Medias Móviles (MACD) de una serie de datos de precios (usualmente los precios de cierre de una acción). El MACD es un indicador técnico de momentum que muestra la relación entre dos medias móviles exponenciales de los precios.

In [207]:
# Definir una función para calcular la Convergencia/Divergencia de Medias Móviles (MACD)
def calcular_macd(datos, ventana_corta=12, ventana_larga=26, ventana_senal=9):
    ema_corta = datos.ewm(span=ventana_corta, adjust=False).mean()
    ema_larga = datos.ewm(span=ventana_larga, adjust=False).mean()
    macd = ema_corta - ema_larga
    senal = macd.ewm(span=ventana_senal, adjust=False).mean()
    return macd, senal

## Calcular MFI


---

### La función calcular_mfi tiene como objetivo calcular el Índice de Flujo de Dinero (MFI) de un activo financiero. El MFI es un indicador técnico de momentum que utiliza el precio y el volumen para medir la presión de compra y venta.

In [208]:
def calcular_mfi(datos, ventana=14):
  """Calcula el Índice de Flujo de Dinero (MFI)."""
  typical_price = (datos['HIGH'] + datos['LOW'] + datos['CLOSE']) / 3
  money_flow = typical_price * datos['VOLUME']
  
  # Calcular el flujo de dinero positivo y negativo
  delta_tp = typical_price.diff()
  positive_flow = money_flow.where(delta_tp > 0, 0)
  negative_flow = money_flow.where(delta_tp < 0, 0)
  
  # Sumar los flujos en la ventana
  positive_flow_sum = positive_flow.rolling(window=ventana).sum()
  negative_flow_sum = negative_flow.rolling(window=ventana).sum()
  
  # Calcular MFR y MFI
  mfr = positive_flow_sum / negative_flow_sum
  mfi = 100 - (100 / (1 + mfr))
  
  return mfi

## Calcular Estocastico



---

### La función calcular_estocastico tiene como objetivo calcular el indicador técnico Estocástico. El Estocástico es un indicador de momentum que compara el precio de cierre de un activo con su rango de precios durante un período específico, generalmente 14 días.


In [209]:
def calcular_estocastico(datos, ventana_k=14, ventana_d=3):
  """Calcula el oscilador estocástico (%K y %D)."""
  lowest_low = datos['LOW'].rolling(window=ventana_k).min()
  highest_high = datos['HIGH'].rolling(window=ventana_k).max()
  k_percent = 100 * ((datos['CLOSE'] - lowest_low) / (highest_high - lowest_low))
  d_percent = k_percent.rolling(window=ventana_d).mean()
  return k_percent, d_percent

# Gráficos de Tendencias


---



## Los gráficos de tendencia proporciona una base para entender la dirección general del precio.



## Gráfico de Velas


---


### Uso: El gráfico de velas muestra la apertura, cierre, máximo y mínimo de cada período de tiempo (día, semana, etc.). Cada "vela" en el gráfico representa la acción del precio durante ese período.

Función en el Análisis Técnico:

Identificar patrones de precios como velas alcistas, bajistas, doji, martillo, etc.
Visualizar la volatilidad del precio mediante el tamaño de las velas (velas largas indican mayor volatilidad).
Detectar posibles cambios en la tendencia del precio mediante la formación de patrones de velas.

In [210]:
# Gráfico de Velas
for ticker in tickers:
    if 'OPEN' in acciones.columns:
        # Crear el gráfico de velas
        grafico_velas = go.Figure(data=[go.Candlestick(x=acciones.index,
                                                       open=acciones['OPEN'],
                                                       high=acciones['HIGH'],
                                                       low=acciones['LOW'],
                                                       close=acciones['CLOSE'])])

        # Actualizar el diseño del gráfico
        grafico_velas.update_layout(xaxis_rangeslider_visible=False, title=f'Gráfico de Velas de {ticker}')
        grafico_velas.update_xaxes(title_text='Fecha')
        grafico_velas.update_yaxes(title_text=f'Precio de Cierre de {ticker}', tickprefix='$')

        # Mostrar el gráfico
        grafico_velas.show()

In [211]:

for ticker in tickers:
    if 'OPEN' in acciones.columns:
        # Crear el gráfico de velas
        c_candlestick = go.Figure(data=[go.Candlestick(x=acciones.index,
                                                            open=acciones['OPEN'],
                                                            high=acciones['HIGH'],
                                                            low=acciones['LOW'],
                                                            close=acciones['CLOSE'])])
        c_candlestick.update_xaxes(
                    title_text='Date',
                    rangeslider_visible=True,
                    rangeselector=dict(
                        buttons=list([
                            dict(count=1, label='1M', step='month', stepmode='backward'),
                            dict(count=6, label='6M', step='month', stepmode='backward'),
                            dict(count=1, label='YTD', step='year', stepmode='todate'),
                            dict(count=1, label='1Y', step='year', stepmode='backward'),
                            dict(step='all')])))

        c_candlestick.update_layout(
                    title={
                        'text': f'{ticker} Customized Candlestick Chart',
                        'y': 0.9,
                        'x': 0.5,
                        'xanchor': 'center',
                        'yanchor': 'top'})

        c_candlestick.update_yaxes(title_text=f'{ticker} Close Price', tickprefix='$')
        c_candlestick.show()

## Gráfico OHLC


---


### Uso: El gráfico OHLC (Open, High, Low, Close) es similar al gráfico de velas, pero con un formato diferente. Muestra la apertura, máximo, mínimo y cierre del precio en forma de barras verticales.

Función en el Análisis Técnico:

Proporciona una visión general rápida de la acción del precio.
Es útil para identificar niveles de soporte y resistencia.
Puede ser más fácil de leer que el gráfico de velas para algunos traders.

In [212]:
# Gráfico OHLC
for ticker in tickers:
    if 'OPEN' in acciones.columns:
        # Crear el gráfico OHLC
        grafico_ohlc = go.Figure(data=[go.Ohlc(x=acciones.index,
                                               open=acciones['OPEN'],
                                               high=acciones['HIGH'],
                                               low=acciones['LOW'],
                                               close=acciones['CLOSE'])])

        # Actualizar el diseño del gráfico
        grafico_ohlc.update_layout(xaxis_rangeslider_visible=False, title=f'Gráfico OHLC de {ticker}')
        grafico_ohlc.update_xaxes(title_text='Fecha')
        grafico_ohlc.update_yaxes(title_text=f'Precio de Cierre de {ticker}', tickprefix='$')

        # Mostrar el gráfico
        grafico_ohlc.show()

In [213]:
# Gráfico OHLC personalizado
for ticker in tickers:
    if 'OPEN' in acciones.columns:
        c_ohlc = go.Figure(data=[go.Ohlc(x=acciones.index,
                                         open=acciones['OPEN'],
                                         high=acciones['HIGH'],
                                         low=acciones['LOW'],
                                         close=acciones['CLOSE'])])

        c_ohlc.update_xaxes(
            title_text='Date',
            rangeslider_visible=True,
            rangeselector=dict(
                buttons=list([
                    dict(count=1, label='1M', step='month', stepmode='backward'),
                    dict(count=6, label='6M', step='month', stepmode='backward'),
                    dict(count=1, label='YTD', step='year', stepmode='todate'),
                    dict(count=1, label='1Y', step='year', stepmode='backward'),
                    dict(step='all')])))

        c_ohlc.update_layout(
            title={
                'text': f'{ticker} Customized OHLC Chart',
                'y': 0.9,
                'x': 0.5,
                'xanchor': 'center',
                'yanchor': 'top'})
        c_ohlc.update_yaxes(title_text=f'{ticker} Close Price', tickprefix='$')
        c_ohlc.show()


## Gráfico de Medias Móviles


---


### Uso: El gráfico de medias móviles muestra la media móvil del precio durante un período de tiempo específico (por ejemplo, 20 días, 50 días, 200 días).

Función en el Análisis Técnico:

Identificar la dirección de la tendencia (alcista, bajista o lateral).
Determinar posibles puntos de entrada y salida del mercado.
Filtrar el "ruido" del mercado y enfocarse en la tendencia principal.

In [214]:
# Gráfico de Medias Móviles
for ticker in tickers:
    if 'CLOSE' in acciones.columns:
        # Calcular las medias móviles de 20, 50 y 200 días
        acciones[f'SMA20_{ticker}'] = acciones['CLOSE'].rolling(window=20).mean()
        acciones[f'SMA50_{ticker}'] = acciones['CLOSE'].rolling(window=50).mean()
        acciones[f'SMA200_{ticker}'] = acciones['CLOSE'].rolling(window=200).mean()

        # Crear el gráfico de medias móviles
        grafico_ma = go.Figure()
        grafico_ma.add_trace(go.Scatter(x=acciones.index, y=acciones['CLOSE'], mode='lines', name=f'Precio de Cierre de {ticker}'))
        grafico_ma.add_trace(go.Scatter(x=acciones.index, y=acciones[f'SMA20_{ticker}'], mode='lines', name='SMA 20 días'))
        grafico_ma.add_trace(go.Scatter(x=acciones.index, y=acciones[f'SMA50_{ticker}'], mode='lines', name='SMA 50 días'))
        grafico_ma.add_trace(go.Scatter(x=acciones.index, y=acciones[f'SMA200_{ticker}'], mode='lines', name='SMA 200 días'))

        # Actualizar el diseño del gráfico
        grafico_ma.update_layout(title=f'Medias Móviles de {ticker}', xaxis_title='Fecha', yaxis_title='Precio')
        grafico_ma.update_yaxes(tickprefix='$')

        # Mostrar el gráfico
        grafico_ma.show()

# Gráficos de Volumen


---

## El gráfico de volumen ayuda a confirmar la fuerza de la tendencia observada en los gráficos de tendencia.


## Gráfico de Volumen


---


### Uso: El gráfico de volumen muestra el volumen de operaciones de un activo durante un período de tiempo específico.

Función en el Análisis Técnico:

Confirmar la fuerza de una tendencia (volumen alto en la dirección de la tendencia).
Identificar posibles puntos de reversión del precio (volumen bajo seguido de un aumento en la dirección opuesta).
Detectar posibles "falsas rupturas" (rupturas de niveles de soporte o resistencia con bajo volumen).

In [215]:
for ticker in tickers:
    if 'VOLUME' in acciones.columns:
        # Crear el gráfico de volumen
        grafico_volumen = go.Figure()
        grafico_volumen.add_trace(go.Bar(x=acciones.index, y=acciones['VOLUME'], name='Volumen', marker_color='darkred'))  # Color rojo vivo

        # Actualizar el diseño del gráfico
        grafico_volumen.update_layout(title=f'Volumen de {ticker}', xaxis_title='Fecha', yaxis_title='Volumen')

        # Mostrar el gráfico
        grafico_volumen.show()

# Gráficos de Fuerza

---



## Los gráficos de fuerza (osciladores) se utilizan para identificar posibles puntos de entrada y salida, así como para detectar divergencias o confirmaciones de la tendencia.

## Gráfico de RSI


---


### Uso: El gráfico de RSI (Índice de Fuerza Relativa) mide la velocidad y el cambio de los movimientos de precios. Se utiliza para identificar condiciones de sobrecompra y sobreventa.

Función en el Análisis Técnico:

Identificar posibles puntos de reversión del precio.
Confirmar la fuerza de una tendencia.
Detectar divergencias entre el precio y el RSI, que pueden indicar un cambio en la tendencia.

In [216]:
# Gráfico de RSI
for ticker in tickers:
    if 'CLOSE' in acciones.columns:
        # Calcular el RSI
        acciones[f'RSI_{ticker}'] = calcular_rsi(acciones['CLOSE'], ventana=14)

        # Crear el gráfico de RSI
        grafico_rsi = go.Figure()
        grafico_rsi.add_trace(go.Scatter(x=acciones.index, y=acciones[f'RSI_{ticker}'], mode='lines', name='RSI'))
        grafico_rsi.add_hline(y=70, line_dash="dash", line_color="red")
        grafico_rsi.add_hline(y=30, line_dash="dash", line_color="green")

        # Actualizar el diseño del gráfico
        grafico_rsi.update_layout(title=f'RSI de {ticker}', xaxis_title='Fecha', yaxis_title='RSI')

        # Mostrar el gráfico
        grafico_rsi.show()

## Gráfico de MACD


---


### Uso: El gráfico de MACD (Convergencia/Divergencia de Medias Móviles) muestra la relación entre dos medias móviles exponenciales del precio. Se utiliza para identificar cambios en la fuerza, dirección, impulso y duración de una tendencia.

Función en el Análisis Técnico:

Detectar cruces entre la línea MACD y la línea de señal, que pueden indicar señales de compra o venta.
Identificar divergencias entre el precio y el MACD, que pueden indicar un cambio en la tendencia.
Confirmar la fuerza de una tendencia.

In [ ]:
# Gráfico de MACD
for ticker in tickers:
    if 'CLOSE' in acciones.columns:
        # Calcular el MACD y la señal
        acciones[f'MACD_{ticker}'], acciones[f'Signal_{ticker}'] = calcular_macd(acciones['CLOSE'])

        # Crear el gráfico de MACD
        grafico_macd = go.Figure()
        grafico_macd.add_trace(go.Scatter(x=acciones.index, y=acciones[f'MACD_{ticker}'], mode='lines', name='MACD'))
        grafico_macd.add_trace(go.Scatter(x=acciones.index, y=acciones[f'Signal_{ticker}'], mode='lines', name='Señal'))

        # Actualizar el diseño del gráfico
        grafico_macd.update_layout(title=f'MACD de {ticker}', xaxis_title='Fecha', yaxis_title='MACD')

        # Mostrar el gráfico
        grafico_macd.show()

## Gráfico de Bandas de Bollinger


---


### Uso: El gráfico de bandas de Bollinger muestra la volatilidad del mercado mediante bandas que se expanden y contraen alrededor del precio.

Función en el Análisis Técnico:

Identificar períodos de alta y baja volatilidad.
Detectar posibles puntos de reversión del precio (cuando el precio toca o supera las bandas).
Confirmar la fuerza de una tendencia (cuando las bandas se expanden en la dirección de la tendencia).

In [ ]:
# Gráfico de Bandas de Bollinger
for ticker in tickers:
    if 'CLOSE' in acciones.columns:
        # Calcular las bandas de Bollinger
        acciones[f'BB_upper_{ticker}'] = acciones['CLOSE'].rolling(window=20).mean() + (acciones['CLOSE'].rolling(window=20).std() * 2)
        acciones[f'BB_lower_{ticker}'] = acciones['CLOSE'].rolling(window=20).mean() - (acciones['CLOSE'].rolling(window=20).std() * 2)

        # Crear el gráfico de bandas de Bollinger
        grafico_bb = go.Figure()
        grafico_bb.add_trace(go.Scatter(x=acciones.index, y=acciones['CLOSE'], mode='lines', name=f'Precio de Cierre de {ticker}'))
        grafico_bb.add_trace(go.Scatter(x=acciones.index, y=acciones[f'BB_upper_{ticker}'], mode='lines', name='Banda Superior'))
        grafico_bb.add_trace(go.Scatter(x=acciones.index, y=acciones[f'BB_lower_{ticker}'], mode='lines', name='Banda Inferior'))

        # Actualizar el diseño del gráfico
        grafico_bb.update_layout(title=f'Bandas de Bollinger de {ticker}', xaxis_title='Fecha', yaxis_title='Precio')
        grafico_bb.update_yaxes(tickprefix='$')

        # Mostrar el gráfico
        grafico_bb.show()

# Índice de Flujo de Dinero (MFI): Midiendo la Presión de Compra y Venta


---


## Uso: El Índice de Flujo de Dinero (MFI) es un indicador de momentum que mide la presión de compra y venta utilizando el precio y el volumen. Se calcula durante un período específico, generalmente 14 días.

Función en el Análisis Técnico:

Identificar condiciones de sobrecompra y sobreventa. Un MFI por encima de 80 generalmente se considera sobrecompra, mientras que un MFI por debajo de 20 se considera sobreventa.
Confirmar la fuerza de una tendencia. Un MFI alto en una tendencia alcista y un MFI bajo en una tendencia bajista confirman la fuerza de la tendencia.
Detectar divergencias entre el precio y el MFI, que pueden indicar un cambio en la tendencia.

In [ ]:
for ticker in tickers:
    if 'CLOSE' in acciones.columns:
        # Calcular el MFI
        acciones[f'MFI_{ticker}'] = calcular_mfi(acciones, ventana=14)

        # Crear el gráfico de MFI
        grafico_mfi = go.Figure()
        grafico_mfi.add_trace(go.Scatter(x=acciones.index, y=acciones[f'MFI_{ticker}'], mode='lines', name='MFI'))
        grafico_mfi.add_hline(y=80, line_dash="dash", line_color="red")  # Sobrecompra
        grafico_mfi.add_hline(y=20, line_dash="dash", line_color="green")  # Sobreventa

        # Actualizar el diseño del gráfico
        grafico_mfi.update_layout(title=f'MFI de {ticker}', xaxis_title='Fecha', yaxis_title='MFI')

        # Mostrar el gráfico
        grafico_mfi.show()

### Análisis Gráfico MFI

El MFI, al igual que el RSI, es un oscilador que se mueve entre 0 y 100, y se utiliza para identificar condiciones de sobrecompra y sobreventa.
En este gráfico, observamos que el MFI oscila entre estos dos extremos a lo largo del período de tiempo, mostrando períodos en los que el activo se considera sobrecomprado (por encima de 80) y sobrevendido (por debajo de 20).
También podemos identificar momentos en los que el MFI confirma la fuerza de una tendencia o señala posibles cambios en la misma.

# Estocástico: Comparando el Precio de Cierre con su Rango


---


##Uso: El Estocástico es un indicador de momentum que compara el precio de cierre de un activo con su rango de precios durante un período específico, generalmente 14 días. Se compone de dos líneas: %K y %D.

Función en el Análisis Técnico:

Identificar condiciones de sobrecompra y sobreventa. Un Estocástico %K por encima de 80 generalmente se considera sobrecompra, mientras que un %K por debajo de 20 se considera sobreventa.
Generar señales de trading. Los cruces de las líneas %K y %D pueden generar señales de compra o venta.
Confirmar la fuerza de una tendencia. Un Estocástico alto en una tendencia alcista y un Estocástico bajo en una tendencia bajista confirman la fuerza de la tendencia.

In [ ]:
for ticker in tickers:
    if 'CLOSE' in acciones.columns:
        # Calcular el Estocástico
        acciones[f'Stochastic_K_{ticker}'], acciones[f'Stochastic_D_{ticker}'] = calcular_estocastico(acciones, ventana_k=14)

        # Crear el gráfico del Estocástico
        grafico_estocastico = go.Figure()
        grafico_estocastico.add_trace(go.Scatter(x=acciones.index, y=acciones[f'Stochastic_K_{ticker}'], mode='lines', name='%K'))
        grafico_estocastico.add_trace(go.Scatter(x=acciones.index, y=acciones[f'Stochastic_D_{ticker}'], mode='lines', name='%D'))
        grafico_estocastico.add_hline(y=80, line_dash="dash", line_color="red")  # Sobrecompra
        grafico_estocastico.add_hline(y=20, line_dash="dash", line_color="green")  # Sobreventa

        # Actualizar el diseño del gráfico
        grafico_estocastico.update_layout(title=f'Estocástico de {ticker}', xaxis_title='Fecha', yaxis_title='Estocástico')

        # Mostrar el gráfico
        grafico_estocastico.show()

### Análisis Gráfico Estocástico

El Estocástico, al igual que el RSI y el MFI, es un oscilador que se mueve entre 0 y 100.
Valores por encima de 80 generalmente se consideran sobrecompra, mientras que valores por debajo de 20 se consideran sobreventa.
En este gráfico, observamos que las líneas %K (línea azul) y %D (línea naranja) del Estocástico oscilan entre estos dos extremos, indicando períodos de sobrecompra y sobreventa.
También podemos identificar momentos en los que el Estocástico ha confirmado la fuerza de una tendencia o ha señalado posibles cambios en la misma.


## Conclusión


---


### Conclusión general del análisis:

El análisis técnico realizado, utilizando gráficos de velas, OHLC, volumen, medias móviles, RSI, MACD, Bandas de Bollinger, MFI y Estocástico, ha proporcionado una visión completa del comportamiento del activo en el mercado durante el período considerado (2020-2025). Se observa una evolución desde una fuerte tendencia alcista inicial hasta una posible reversión bajista, con fases de consolidación y recuperación intermedias. Los diferentes indicadores y gráficos confirman las distintas etapas del movimiento del precio, y señalan posibles puntos de entrada y salida para las operaciones. Sin embargo, es importante recordar que el análisis técnico no es infalible y que debe combinarse con el análisis fundamental y otros factores para tomar decisiones de inversión informadas.
Los indicadores y gráficos presentados son solo una muestra de las herramientas disponibles. Explora otros indicadores y estrategias.
El trading conlleva riesgos y es importante gestionar tu riesgo de manera responsable.

### Glosario por gráfico:

Gráfico de velas/OHLC: Muestra la evolución del precio a lo largo del tiempo, desde una tendencia alcista inicial hasta una posible reversión bajista. Los patrones de velas/barras OHLC pueden ayudar a identificar posibles puntos de entrada y salida.

Gráfico de volumen: El volumen confirma la fuerza de las tendencias y las reversiones. Un volumen alto durante las subidas indica fuerza alcista, mientras que un volumen alto durante las caídas indica fuerza bajista.

Gráfico de medias móviles: Las medias móviles ayudan a identificar la dirección de la tendencia y posibles puntos de soporte y resistencia. Los cruces de medias móviles pueden generar señales de compra o venta.

Gráfico RSI: El RSI indica el momentum del precio y posibles condiciones de sobrecompra y sobreventa. Las divergencias entre el precio y el RSI pueden señalar cambios en la tendencia.

Gráfico MACD: El MACD confirma la fuerza y la dirección de la tendencia. Los cruces del MACD y la línea de señal, así como las divergencias, pueden generar señales de trading.

Gráfico Bandas de Bollinger: Las Bandas de Bollinger muestran la volatilidad del mercado. Las rupturas de las bandas pueden indicar una continuación o un cambio de la tendencia.

Gráfico MFI: El MFI mide la presión de compra y venta. Las señales de sobrecompra y sobreventa pueden ayudar a identificar posibles puntos de reversión.

Gráfico Estocástico: El Estocástico también indica el momentum del precio y posibles condiciones de sobrecompra y sobreventa. Los cruces de las líneas %K y %D pueden generar señales de trading.
Consideraciones finales:
